## Setting Up:

In [2]:
import pandas as pd
import os
import warnings
from tqdm.notebook import tqdm
import zipfile
import cdsapi
import numpy as np
import glob
import xarray as xr

from scripts.config_NOR import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

## Download monthly ERA5-Land variables:

In [2]:
RUN = False
if RUN:
    os.makedirs(path_ERA5_raw, exist_ok=True)
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land-monthly-means', {
            'product_type': ['monthly_averaged_reanalysis'],
            'variable': [
                '10m_u_component_of_wind',
                '10m_v_component_of_wind',
                '2m_temperature',
                'forecast_albedo',
                'snow_cover',
                'snow_density',
                'snow_depth_water_equivalent',
                'snowfall',
                'snowmelt',
                'surface_latent_heat_flux',
                'surface_net_thermal_radiation',
                'surface_sensible_heat_flux',
                'surface_solar_radiation_downwards',
                'total_precipitation',
            ],
            'year': [
                '1950',
                '1951',
                '1952',
                '1953',
                '1954',
                '1955',
                '1956',
                '1957',
                '1958',
                '1959',
                '1960',
                '1961',
                '1962',
                '1963',
                '1964',
                '1965',
                '1966',
                '1967',
                '1968',
                '1969',
                '1970',
                '1971',
                '1972',
                '1973',
                '1974',
                '1975',
                '1976',
                '1977',
                '1978',
                '1979',
                '1980',
                '1981',
                '1982',
                '1983',
                '1984',
                '1985',
                '1986',
                '1987',
                '1988',
                '1989',
                '1990',
                '1991',
                '1992',
                '1993',
                '1994',
                '1995',
                '1996',
                '1997',
                '1998',
                '1999',
                '2000',
                '2001',
                '2002',
                '2003',
                '2004',
                '2005',
                '2006',
                '2007',
                '2008',
                '2009',
                '2010',
                '2011',
                '2012',
                '2013',
                '2014',
                '2015',
                '2016',
                '2017',
                '2018',
                '2019',
                '2020',
                '2021',
                '2022',
                '2023',
                '2024',
            ],
            'month': [
                '01',
                '02',
                '03',
                '04',
                '05',
                '06',
                '07',
                '08',
                '09',
                '10',
                '11',
                '12',
            ],
            'time': ['00:00'],
            "data_format": "netcdf",
            "download_format": "zip",
            'area': [
                72,    # North latitude
                4,     # West longitude
                57,    # South latitude
                32,    # East longitude
            ],
        }, path_ERA5_raw+'download.netcdf.zip')
    with zipfile.ZipFile(path_ERA5_raw+'download.netcdf.zip', 'r') as zip:
        zip.extractall(path_ERA5_raw)
    c.retrieve("reanalysis-era5-single-levels", {
            "product_type": ["reanalysis"],
            "variable": ["geopotential"],
            "year": ["2024"],
            "month": ["06"],
            "day": ["01"],
            "time": ["12:00"],
            "data_format": "netcdf",
            'area': [
                72,    # North latitude
                4,     # West longitude
                57,    # South latitude
                32,    # East longitude
            ],
        }, path_ERA5_raw+'era5_geopotential_pressure.nc')

In [14]:
! ls "{path_ERA5_raw}"

data_stream-moda.nc  era5_geopotential_pressure.nc
download.netcdf.zip  era5_monthly_averaged_data.nc


In [11]:
dc=xr.open_dataset(path_ERA5_raw+'data_stream-moda.nc')

In [12]:
dc2 = dc.rename({'valid_time': 'time'}) # Coordinates have changed recently in the API, this is to keep compatibility with our code

In [13]:
dc2.to_netcdf(path_ERA5_raw + "era5_monthly_averaged_data.nc")

In [16]:
display(xr.open_dataset(path_ERA5_raw+'era5_monthly_averaged_data.nc'))

display(xr.open_dataset('/home/mburlet/scratch/data/DATA_MB/ERA5Land/raw/era5_monthly_averaged_data_scratch.nc'))

display(xr.open_dataset(path_ERA5_raw+'era5_geopotential_pressure.nc'))

display(xr.open_dataset('/home/mburlet/scratch/data/DATA_MB/ERA5Land/raw/era5_geopotential_pressure_scratch.nc'))

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 900, latitude: 151, longitude: 281)
Coordinates:
    number     int64 8B ...
  * time       (time) datetime64[ns] 7kB 1950-01-01 1950-02-01 ... 2024-12-01
  * latitude   (latitude) float64 1kB 72.0 71.9 71.8 71.7 ... 57.2 57.1 57.0
  * longitude  (longitude) float64 2kB 4.0 4.1 4.2 4.3 ... 31.7 31.8 31.9 32.0
    expver     (time) <U4 14kB ...
Data variables: (12/14)
    u10        (time, latitude, longitude) float32 153MB ...
    v10        (time, latitude, longitude) float32 153MB ...
    t2m        (time, latitude, longitude) float32 153MB ...
    fal        (time, latitude, longitude) float32 153MB ...
    snowc      (time, latitude, longitude) float32 153MB ...
    rsn        (time, latitude, longitude) float32 153MB ...
    ...         ...
    smlt       (time, latitude, longitude) float32 153MB ...
    slhf       (time, latitude, longitude) float32 153MB ...
    str        (time, latitude, longitude) float32 153MB ...
    sshf       (time, latitude, longitude) float32 153MB ...
    ssrd       (time, latitude, longitude) float32 153MB ...
    tp         (time, latitude, longitude) float32 153MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T19:53 GRIB to CDM+CF via cfgrib-0.9.1...

<xarray.Dataset> Size: 2GB
Dimensions:    (longitude: 201, latitude: 51, expver: 2, time: 895)
Coordinates:
  * longitude  (longitude) float32 804B 0.0 0.1 0.2 0.3 ... 19.7 19.8 19.9 20.0
  * latitude   (latitude) float32 204B 50.0 49.9 49.8 49.7 ... 45.2 45.1 45.0
  * expver     (expver) int32 8B 1 5
  * time       (time) datetime64[ns] 7kB 1950-01-01 1950-02-01 ... 2024-07-01
Data variables: (12/14)
    u10        (time, expver, latitude, longitude) float64 147MB ...
    v10        (time, expver, latitude, longitude) float64 147MB ...
    t2m        (time, expver, latitude, longitude) float64 147MB ...
    fal        (time, expver, latitude, longitude) float64 147MB ...
    snowc      (time, expver, latitude, longitude) float64 147MB ...
    rsn        (time, expver, latitude, longitude) float64 147MB ...
    ...         ...
    smlt       (time, expver, latitude, longitude) float64 147MB ...
    slhf       (time, expver, latitude, longitude) float64 147MB ...
    str        (time, expver, latitude, longitude) float64 147MB ...
    sshf       (time, expver, latitude, longitude) float64 147MB ...
    ssrd       (time, expver, latitude, longitude) float64 147MB ...
    tp         (time, expver, latitude, longitude) float64 147MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-08-08 11:30:10 GMT by grib_to_netcdf-2.24.0: /opt/ecmw...

<xarray.Dataset> Size: 29kB
Dimensions:     (valid_time: 1, latitude: 61, longitude: 113)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 8B 2024-06-01T12:00:00
  * latitude    (latitude) float64 488B 72.0 71.75 71.5 ... 57.5 57.25 57.0
  * longitude   (longitude) float64 904B 4.0 4.25 4.5 4.75 ... 31.5 31.75 32.0
    expver      <U4 16B ...
Data variables:
    z           (valid_time, latitude, longitude) float32 28kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T23:38 GRIB to CDM+CF via cfgrib-0.9.1...

<xarray.Dataset> Size: 52MB
Dimensions:    (longitude: 3600, latitude: 1801, time: 1)
Coordinates:
  * longitude  (longitude) float32 14kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
  * latitude   (latitude) float32 7kB 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 8B 2013-08-09T12:00:00
Data variables:
    z          (time, latitude, longitude) float64 52MB ...
Attributes:
    Conventions:               CF-1.6
    history:                   Fri Jun 12 14:41:56 2020: ncpdq -U geo_1279l4_...
    NCO:                       4.7.2
    nco_openmp_thread_number:  1